这个文件检查怎么样保存和恢复通过Estimator创建的TensorFlow models。 TensorFlow提供两个模型格式：
* checkpoints: 格式依赖于创建模型的代码
* SavedModel: 格式依赖于创建模型的代码
这个文档专注于checkpoints。 SavedModel的详细信息见[Saving and Restoring](https://www.tensorflow.org/programmers_guide/saved_model?hl=zh-cn)

## Saving partially-trained models
Estimators 自动将下面的东西写到磁盘：
* __checkpoints__ 在training的时候创建的模型。
* __event files__ TensorBoard用来创造visualizations的信息。

指定Estimator top-level目录储存信息，通过给Estimator的构造器的可选参数 __model_dir__ 赋值。例如，以下的代码将 __model_dir='models/iris'__ ,例如下面例1：

如果调用 Estimator的train方法，例如下面示例2：

如下面的图形表示的，train第一次调用增加checkpoints和其他的文件到 __model_dir__ 目录

![Title](./first_train_calls.png)

In [ ]:
# 1:
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,hidden_units=[10,10],
    n_classes=3, model_dir ='models/iris')

# 2:
classifier.train(input_fn=lambda:train_input_fn(train_x,train_y,batch_size=100),
                steps = 200)

## model_dir中创建的文件，start of training is 1, end of training is 200
$ ls -1 models/iris # unix command
checkpoint
events.out.tfevents.timestamp.hostname
graph.pbtxt
model.ckpt-1.data-00000-of-00001
model.ckpt-1.index
model.ckpt-1.meta
model.ckpt-200.data-00000-of-00001
model.ckpt-200.index
model.ckpt-200.meta

### Default checkpoint directory
如果不指定Estimator中 __model_dir__ 的参数，那么checkpoint会写到Python选择的 [tempfile.mkdtemp](https://docs.python.org/3/library/tempfile.html#tempfile.mkdtemp) 函数的临时目录当中。 __tempfile.mkdtemp__ 函数选择一个合适与你操作系统的安全的合适的临时目录。例如， macOs可能创建的目录如下所示：

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3)

print(classifier.model_dir)
# /var/folders/0s/5q9kfzfj3gx2knj0vj8p68yc00dhcr/T/tmpYm1Rwa

### Checkpointing Frequency
默认情况下， Estimator保存checkpoints到 __model_dir__ 当中依照以下过程：
* 每十分钟(600s)写一个checkpoint
* 当train 方法开始的时候(第一次迭代）和完成的时候（最后一次迭代）写一个checkpoint。
* 在目录中保留最近的5个checkpoints

可以通过以下步骤修改默认的schedule：
1. 创造一个 __[RunConfig](https://www.tensorflow.org/api_docs/python/tf/estimator/RunConfig?hl=zh-cn)__ 对象定义想要的schedule
2. 当实例化Estimator时，将 __RunConfig__ 对象传递给Estimator中的 __config__ 参数。  

例如，以下的代码修改checkpointing schedule到每20分钟保存一次并且保留最近的10个checkpoints：

In [ ]:
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs = 20*60 # Save checkpoints every 20 minutes.
    keep_checkpoint_max = 10,     # Retain the 10 most recent checkpoints.
)

classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    hidden_units=[10,10],
    n_classes=3,
    model_dir = 'models/iris',
    config=my_checkpointing_config
)

## Restoring your model
第一次调用 Estimator train方法的时候， TensorFlow储存一个checkpoint到 __model_dir__ . 接下来每次调用Estimator的train，evaluate，predict方法会导致下面的：
1. Estimator通过调用 __model_fn()__ 来创建模型的 graph
2. Estimator 从最近储存的checkpoint数据中初始化新模型的weights

也就是说，就像下面的图片显示，一旦checkpoints存在，TensorFlow会在调用 __train(),evaluation(),predict()__ 的时候重建模型。
![Title](./subsequent_calls.png)

#### Avoiding a bad restoration
只有当模型和checkpoint兼容的时候，从一个checkpoint恢复(restore)一个模型才能够工作。
以下示例当中会出现重载错误：


In [1]:
## 首先训练一个两层hidden layer并且units=10
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 10],
    n_classes=3,
    model_dir='models/iris')

classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, batch_size=100),
        steps=200)

## 然后修改hidden_units参数到[20,20]
classifier2 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[20, 20],  # Change the number of neurons in the model.
    n_classes=3,
    model_dir='models/iris')

classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, batch_size=100),
        steps=200)

## 由于第二个模型的参数和第一个模型的参数不匹配，所以，会得到一个错误
...
InvalidArgumentError (see above for traceback): tensor_name =
dnn/hiddenlayer_1/bias/t_0/Adagrad; shape in shape_and_slice spec [10]
does not match the shape stored in checkpoint: [20]

## 所以修改参数的时候，必须要修改model_dir的值，这样保证checkpoints时可恢复的


SyntaxError: invalid syntax (<ipython-input-1-4fb4524e9f5b>, line 25)

## Summary
Checkpoints由Estimator创建提供一个可以自动保存和恢复模型的机制。
查看[Saving and Restoring](https://www.tensorflow.org/programmers_guide/saved_model?hl=zh-cn) 来获得更加详细的信息：
* 怎么样在 low-level TensorFlow API 上面保存和恢复model
* 用 SavedModel的格式导出和导入模型，这是一个语言中立，可恢复的，序列化的格式。